<a href="https://colab.research.google.com/github/bitanb1999/Summer-Analytics-2021/blob/main/SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Summer Analytics Capstone Project

####Pre-requisites

In [33]:
import numpy as np
import scipy.stats
import scipy.special
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

#Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

#For Missing Value and Feature Engineering
from sklearn.feature_selection import SelectKBest, chi2, f_classif, VarianceThreshold
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier

import time
import seaborn as sns
%matplotlib inline

In [18]:
## getting the data
train=pd.read_csv("/content/Train_Data.csv")
test=pd.read_csv("/content/Test_Data.csv")

In [4]:
train.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71


In [5]:
test.head()

,date,campaign,adgroup,ad,cost,impressions,clicks,conversions
0,01-03-2021,campaign 1,adgroup 1,ad 1,0.58,121,49,1
1,01-03-2021,campaign 1,adgroup 3,ad 1,0.17,22,12,0
2,01-03-2021,campaign 1,adgroup 4,ad 1,0.05,5,3,0
3,01-03-2021,campaign 1,adgroup 2,ad 1,0.01,2,1,0
4,01-03-2021,campaign 1,adgroup 2,ad 2,0.01,3,1,0


In [6]:
#helper function to view the table content details
from IPython.display import display, HTML

def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )
def tbl_report(tbl, cols=None, card=52):
    print("Table Shape", tbl.shape)
    dtypes = tbl.dtypes
    nulls = []
    uniques = []
    numuniques = []
    vcs = []
    for col in dtypes.index:
        n = tbl[col].isnull().sum()
        nulls.append(n)
        strdtcol = str(dtypes[col])
        #if strdtcol == 'object' or strdtcol[0:3] == 'int' or strdtcol[0:3] == 'int':
        #print(strdtcol)
        uniqs = tbl[col].unique()
        uniquenums = uniqs.shape[0]
        if uniquenums < card: # low cardinality
            valcounts = pd.value_counts(tbl[col], dropna=False)
            vc = "\n".join(["{}:{}".format(k,v) for k, v in valcounts.items()])
        else:
            vc='NA'
        uniques.append(uniqs)
        numuniques.append(uniquenums)
        vcs.append(vc)
    nullseries = pd.Series(nulls, index=dtypes.index)
    uniqueseries = pd.Series(uniques, index=dtypes.index)
    numuniqueseries = pd.Series(numuniques, index=dtypes.index)
    vcseries = pd.Series(vcs, index=dtypes.index)
    df = pd.concat([dtypes, nullseries, uniqueseries, numuniqueseries, vcseries], axis=1)
    df.columns = ['dtype', 'nulls', 'uniques', 'num_uniques', 'value_counts']
    if cols:
        return pretty_print(df[cols])
    return pretty_print(df)

In [7]:
tbl_report(train)

Table Shape (4571, 9)


,dtype,nulls,uniques,num_uniques,value_counts
date,object,0,"[01-08-2020, 02-08-2020, 03-08-2020, 04-08-2020, 05-08-2020, 06-08-2020, 07-08-2020, 08-08-2020, 09-08-2020, 10-08-2020, 11-08-2020, 12-08-2020, 13-08-2020, 14-08-2020, 15-08-2020, 16-08-2020, 17-08-2020, 18-08-2020, 19-08-2020, 20-08-2020, 21-08-2020, 22-08-2020, 23-08-2020, 24-08-2020, 25-08-2020, 26-08-2020, 27-08-2020, 28-08-2020, 29-08-2020, 30-08-2020, 31-08-2020, 01-09-2020, 02-09-2020, 03-09-2020, 04-09-2020, 05-09-2020, 06-09-2020, 07-09-2020, 08-09-2020, 09-09-2020, 10-09-2020, 11-09-2020, 12-09-2020, 13-09-2020, 14-09-2020, 15-09-2020, 16-09-2020, 17-09-2020, 18-09-2020, 19-09-2020, 20-09-2020, 21-09-2020, 22-09-2020, 23-09-2020, 24-09-2020, 25-09-2020, 26-09-2020, 27-09-2020, 28-09-2020, 29-09-2020, 30-09-2020, 01-10-2020, 02-10-2020, 03-10-2020, 04-10-2020, 05-10-2020, 06-10-2020, 07-10-2020, 08-10-2020, 09-10-2020, 10-10-2020, 11-10-2020, 12-10-2020, 13-10-2020, 14-10-2020, 15-10-2020, 16-10-2020, 17-10-2020, 18-10-2020, 19-10-2020, 20-10-2020, 21-10-2020, 22-10-2020, 23-10-2020, 24-10-2020, 25-10-2020, 26-10-2020, 27-10-2020, 28-10-2020, 29-10-2020, 30-10-2020, 31-10-2020, 01-11-2020, 02-11-2020, 03-11-2020, 04-11-2020, 05-11-2020, 06-11-2020, 07-11-2020, 08-11-2020, ...]",212,NA
campaign,object,0,[campaign 1],1,campaign 1:4571
adgroup,object,0,"[adgroup 1, adgroup 2, adgroup 3, adgroup 4]",4,adgroup 3:1672adgroup 1:1314adgroup 4:858adgroup 2:727
ad,object,0,"[ad 1, ad 2, ad 3, ad 4, ad 5, ad 6, ad 7, ad 8, ad 9, ad 10, ad 11, ad 12, ad 13, ad 14, ad 15, ad 16, ad 17, ad 18, ad 19, ad 20, ad 21, ad 22, ad 23, ad 24, ad 25, ad 26, ad 27, ad 28, ad 29, ad 30, ad 31, ad 32, ad 33, ad 34, ad 35, ad 36, ad 37, ad 38, ad 39, ad 40, ad 41, ad 42, ad 43, ad 44, ad 45, ad 46, ad 47, ad 48, ad 49, ad 50, ad 51, ad 52, ad 53, ad 54, ad 55, ad 56, ad 57, ad 58, ad 59, ad 60, ad 61, ad 62, ad 63, ad 64, ad 65, ad 66, ad 67, ad 68, ad 69, ad 70]",70,NA
impressions,int64,0,"[24, 1, 13, 5, 247, 7, 6, 129, 104, 4, 39, 17, 253, 16, 26, 250, 261, 329, 2, 21, 197, 11, 105, 62, 10, 258, 31, 243, 212, 3, 245, 12, 49, 304, 9, 140, 116, 68, 22, 218, 138, 15, 19, 40, 272, 131, 128, 54, 297, 25, 216, 173, 144, 8, 309, 163, 142, 55, 20, 278, 34, 18, 256, 44, 443, 148, 201, 52, 14, 282, 60, 489, 158, 190, 48, 291, 524, 206, 66, 348, 45, 440, 169, 241, 71, 311, 23, 496, 89, 70, 280, 75, 324, 81, 442, 117, 152, 61, 289, 47, ...]",571,NA
clicks,int64,0,"[6, 0, 4, 126, 2, 3, 52, 56, 5, 16, 128, 10, 114, 123, 130, 8, 112, 1, 68, 40, 139, 9, 14, 117, 79, 140, 13, 7, 156, 60, 46, 39, 129, 95, 73, 78, 18, 37, 54, 21, 161, 11, 122, 64, 67, 155, 84, 28, 150, 15, 138, 211, 71, 27, 154, 250, 76, 59, 30, 179, 23, 136, 237, 96, 31, 119, 12, 203, 75, 33, 148, 35, 146, 36, 229, 92, 132, 34, 149, 210, 38, 85, 32, 135, 22, 165, 61, 118, 218, 53, 17, 125, 24, 191, 86, 29, 47, 93, 109, 142, ...]",355,NA
cost,float64,0,"[0.08, 0.0, 0.04, 1.29, 0.05, 0.58, 0.56, 0.22, 0.06, 1.32, 0.07, 0.1, 1.17, 1.27, 1.4, 0.02, 0.09, 0.18, 1.18, 0.01, 0.69, 0.46, 0.44, 1.43, 0.15, 1.21, 0.86, 1.44, 0.13, 1.75, 0.64, 0.03, 0.41, 1.38, 1.04, 0.78, 0.88, 0.21, 1.54, 0.62, 0.23, 1.72, 0.12, 1.33, 0.75, 0.85, 1.73, 0.9, 0.42, 0.35, 1.61, 0.11, 0.16, 1.52, 0.28, 2.48, 0.74, 0.34, 1.7, 0.32, 1.42, 2.69, 0.66, 0.33, 1.9, 0.26, 0.14, 1.46, 2.81, 1.09, 1.28, 1.58, 2.36, 0.82, 0.36, 1.53, 0.37, 0.4, 0.98, 0.3, 1.41, 1.59, 0.29, 2.29, 0.43, 1.0, 0.27, 1.87, 0.71, 1.3, 0.38, 2.41, 0.59, 0.2, 2.12, 0.97, 0.77, 0.52, 1.05, 1.5, ...]",810,NA
conversions,int64,0,"[0, 4, 2, 3, 9, 1, 5, 6, 7, 8, 10, 19, 12, 16, 23, 22, 15, 14, 24, 36, 18, 40, 61, 47, 82, 91, 11, 17, 44, 27, 28, 74, 71, 63, 42, 37, 85, 13, 39, 48, 56, 68, 94, 43, 79, 59, 34, 26, 52]",49,0:32131:5422:2343:1174:1056:795:767:489:358:2810:1112:711:714:723:619:513:415:322:327:318:317:316:236:282:240:247:226:152:171:163:194:124:128:144:148:168:156:134:159:143:174:137:161:185:139:142:179:191:1
revenue,float64,0,"[0.0, 925.71, 339.83, 384.22, 337.57, 397.59, 1476.67, 188.83, 2

In [8]:
tbl_report(test)

Table Shape (318, 8)


,dtype,nulls,uniques,num_uniques,value_counts
date,object,0,"[01-03-2021, 02-03-2021, 03-03-2021, 04-03-2021, 05-03-2021, 06-03-2021, 07-03-2021, 08-03-2021, 09-03-2021, 10-03-2021, 11-03-2021, 12-03-2021, 13-03-2021, 14-03-2021, 15-03-2021]",15,03-03-2021:2808-03-2021:2613-03-2021:2515-03-2021:2509-03-2021:2512-03-2021:2411-03-2021:2414-03-2021:2410-03-2021:2305-03-2021:2102-03-2021:1901-03-2021:1907-03-2021:1206-03-2021:1204-03-2021:11
campaign,object,0,[campaign 1],1,campaign 1:318
adgroup,object,0,"[adgroup 1, adgroup 3, adgroup 4, adgroup 2]",4,adgroup 3:111adgroup 1:90adgroup 2:63adgroup 4:54
ad,object,0,"[ad 1, ad 2, ad 3, ad 4, ad 5, ad 6, ad 7, ad 8, ad 10, ad 71, ad 72, ad 73, ad 74, ad 75, ad 56, ad 55]",16,ad 3:44ad 1:42ad 7:33ad 2:30ad 56:24ad 72:22ad 55:19ad 10:13ad 73:13ad 74:12ad 5:11ad 4:11ad 71:11ad 8:11ad 75:11ad 6:11
cost,float64,0,"[0.58, 0.17, 0.05, 0.01, 0.03, 1.67, 0.0, 1.06, 0.08, 1.64, 0.1, 0.88, 0.02, 0.3, 0.16, 0.98, 0.22, 0.24, 0.65, 1.57, 1.65, 1.36, 0.07, 0.99, 0.46, 0.13, 0.04, 0.94, 0.12, 0.09, 0.18, 0.34, 0.4, 0.14, 0.25, 0.87, 0.81, 0.06, 2.05, 0.67, 1.69, 0.56, 1.59, 1.66, 1.54, 0.91, 1.4, 0.7, 1.07, 0.2, 0.49, 1.94, 2.9, 0.72, 0.32, 0.31, 0.27, 0.55, 0.66, 0.61, 3.36, 2.67, 0.45, 0.78, 0.63, 3.53, 3.47, 0.77, 0.47, 0.62, 1.49, 1.62, 1.21, 1.03, 0.15, 0.44, 0.9, 1.29, 1.15, 0.21, 0.93, 0.29, 0.37, 0.85, 1.16, 1.0, 0.43, 1.17, 1.27, 1.41, 0.97, 0.19, 1.08, 1.39, 0.26, 0.28, 0.11, 0.73, 1.58, 0.41, ...]",109,NA
impressions,int64,0,"[121, 22, 5, 2, 3, 4, 276, 6, 162, 294, 15, 152, 94, 25, 191, 24, 27, 112, 281, 1, 298, 196, 7, 31, 12, 151, 103, 23, 178, 29, 13, 42, 45, 68, 20, 9, 127, 192, 316, 134, 306, 91, 258, 347, 269, 173, 67, 263, 123, 14, 238, 40, 62, 10, 362, 418, 137, 66, 54, 48, 75, 106, 116, 502, 558, 77, 131, 125, 111, 560, 589, 153, 161, 8, 260, 308, 167, 35, 207, 16, 108, 201, 210, 228, 18, 139, 30, 26, 174, 78, 169, 166, 248, 113, 37, 165, 32, 60, 28, 85, ...]",128,NA
clicks,int64,0,"[49, 12, 3, 1, 2, 123, 0, 74, 5, 110, 14, 7, 82, 30, 79, 18, 48, 126, 122, 97, 4, 88, 39, 11, 75, 10, 8, 21, 27, 13, 6, 22, 69, 64, 145, 55, 125, 40, 121, 137, 119, 77, 31, 115, 54, 16, 38, 155, 215, 61, 28, 24, 51, 236, 232, 32, 45, 63, 52, 261, 276, 34, 59, 135, 136, 95, 93, 15, 89, 44, 84, 94, 96, 76, 9, 41, 90, 62, 86, 102, 85, 103, 17, 83, 81, 91, 57, 68, 26, 92, 114, 19, 37, 67, 116, 134, 73, 80]",98,NA
conversions,int64,0,"[1, 0, 3, 2, 4, 6, 8, 14, 13, 15, 5]",11,0:2111:502:263:134:86:35:315:114:113:18:1


In [10]:
train.columns[4:-1]

Index(['impressions', 'clicks', 'cost', 'conversions'], dtype='object')

In [25]:
#making the training and the testing datasets based on the given values
df=train[train.columns[4:-1]]
df.head()

,impressions,clicks,cost,conversions
0,24,6,0.08,0
1,1,0,0.00,0
2,13,4,0.04,0
3,5,4,0.08,0
4,247,126,1.29,4


In [26]:
dftest=test[train.columns[4:-1]]
dftest.head()

,impressions,clicks,cost,conversions
0,121,49,0.58,1
1,22,12,0.17,0
2,5,3,0.05,0
3,2,1,0.01,0
4,3,1,0.01,0


Efficiency metrics (optional - can be used for creating new features/feature engineering): 

The below features can be newly created/calculated and used while building the models. These are based on raw metrics. 

CTR - Clicks / Impression (higher the better - used to evaluate if the users find the ad relevant)

CPC - Cost / Click (lower the better - used to evaluate if the cost for getting a click)

CPA - Cost / Conversion (lower the better - used to evaluate if the cost for getting a conversion)

ROI - Revenue/Cost (higher the better - used to evaluate the effectiveness of the advertising budget spent

In [27]:
#feature engineering
df['ctr']=df['clicks']/df['impressions']

dftest['ctr']=dftest['clicks']/dftest['impressions']



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [28]:
df.head()

,impressions,clicks,cost,conversions,ctr
0,24,6,0.08,0,0.250000
1,1,0,0.00,0,0.000000
2,13,4,0.04,0,0.307692
3,5,4,0.08,0,0.800000
4,247,126,1.29,4,0.510121


In [29]:
dftest.head()

,impressions,clicks,cost,conversions,ctr
0,121,49,0.58,1,0.404959
1,22,12,0.17,0,0.545455
2,5,3,0.05,0,0.600000
3,2,1,0.01,0,0.500000
4,3,1,0.01,0,0.333333


In [30]:
tbl_report(df)

Table Shape (4571, 5)


,dtype,nulls,uniques,num_uniques,value_counts
impressions,int64,0,"[24, 1, 13, 5, 247, 7, 6, 129, 104, 4, 39, 17, 253, 16, 26, 250, 261, 329, 2, 21, 197, 11, 105, 62, 10, 258, 31, 243, 212, 3, 245, 12, 49, 304, 9, 140, 116, 68, 22, 218, 138, 15, 19, 40, 272, 131, 128, 54, 297, 25, 216, 173, 144, 8, 309, 163, 142, 55, 20, 278, 34, 18, 256, 44, 443, 148, 201, 52, 14, 282, 60, 489, 158, 190, 48, 291, 524, 206, 66, 348, 45, 440, 169, 241, 71, 311, 23, 496, 89, 70, 280, 75, 324, 81, 442, 117, 152, 61, 289, 47, ...]",571,NA
clicks,int64,0,"[6, 0, 4, 126, 2, 3, 52, 56, 5, 16, 128, 10, 114, 123, 130, 8, 112, 1, 68, 40, 139, 9, 14, 117, 79, 140, 13, 7, 156, 60, 46, 39, 129, 95, 73, 78, 18, 37, 54, 21, 161, 11, 122, 64, 67, 155, 84, 28, 150, 15, 138, 211, 71, 27, 154, 250, 76, 59, 30, 179, 23, 136, 237, 96, 31, 119, 12, 203, 75, 33, 148, 35, 146, 36, 229, 92, 132, 34, 149, 210, 38, 85, 32, 135, 22, 165, 61, 118, 218, 53, 17, 125, 24, 191, 86, 29, 47, 93, 109, 142, ...]",355,NA
cost,float64,0,"[0.08, 0.0, 0.04, 1.29, 0.05, 0.58, 0.56, 0.22, 0.06, 1.32, 0.07, 0.1, 1.17, 1.27, 1.4, 0.02, 0.09, 0.18, 1.18, 0.01, 0.69, 0.46, 0.44, 1.43, 0.15, 1.21, 0.86, 1.44, 0.13, 1.75, 0.64, 0.03, 0.41, 1.38, 1.04, 0.78, 0.88, 0.21, 1.54, 0.62, 0.23, 1.72, 0.12, 1.33, 0.75, 0.85, 1.73, 0.9, 0.42, 0.35, 1.61, 0.11, 0.16, 1.52, 0.28, 2.48, 0.74, 0.34, 1.7, 0.32, 1.42, 2.69, 0.66, 0.33, 1.9, 0.26, 0.14, 1.46, 2.81, 1.09, 1.28, 1.58, 2.36, 0.82, 0.36, 1.53, 0.37, 0.4, 0.98, 0.3, 1.41, 1.59, 0.29, 2.29, 0.43, 1.0, 0.27, 1.87, 0.71, 1.3, 0.38, 2.41, 0.59, 0.2, 2.12, 0.97, 0.77, 0.52, 1.05, 1.5, ...]",810,NA
conversions,int64,0,"[0, 4, 2, 3, 9, 1, 5, 6, 7, 8, 10, 19, 12, 16, 23, 22, 15, 14, 24, 36, 18, 40, 61, 47, 82, 91, 11, 17, 44, 27, 28, 74, 71, 63, 42, 37, 85, 13, 39, 48, 56, 68, 94, 43, 79, 59, 34, 26, 52]",49,0:32131:5422:2343:1174:1056:795:767:489:358:2810:1112:711:714:723:619:513:415:322:327:318:317:316:236:282:240:247:226:152:171:163:194:124:128:144:148:168:156:134:159:143:174:137:161:185:139:142:179:191:1
ctr,float64,0,"[0.25, 0.0, 0.3076923076923077, 0.8, 0.5101214574898786, 0.2857142857142857, 0.5, 0.40310077519379844, 0.5384615384615384, 0.8333333333333334, 0.41025641025641024, 0.35294117647058826, 0.5059288537549407, 0.375, 0.38461538461538464, 0.4, 0.456, 0.47126436781609193, 0.3951367781155015, 1.0, 0.38095238095238093, 0.7619047619047619, 0.3333333333333333, 0.5685279187817259, 0.45454545454545453, 0.5271317829457365, 0.6451612903225806, 0.5387596899224806, 0.5294117647058824, 0.45161290322580644, 0.48148148148148145, 0.2, 0.37264150943396224, 0.6666666666666666, 0.5714285714285714, 0.2653061224489796, 0.5131578947368421, 0.42857142857142855, 0.39655172413793105, 0.5735294117647058, 0.36363636363636365, 0.43577981651376146, 0.5289855072463768, 0.39593908629441626, 0.3157894736842105, 0.45, 0.5110294117647058, 0.2824427480916031, 0.421875, 0.3888888888888889, 0.5420875420875421, 0.44, 0.5648148148148148, 0.3699421965317919, 0.4652777777777778, 0.358974358974359, 0.5016181229773463, 0.5153374233128835, 0.2605633802816901, 0.509090909090909, 0.539568345323741, 0.875, 0.4411764705882353, 0.5390625, 0.47058823529411764, 0.4772727272727273, 0.3, 0.4762979683972912, 0.4797297297297297, 0.5192307692307693, 0.35714285714285715, 0.5185185185185185, 0.4444444444444444, 0.46099290780141844, 0.47368421052631576, 0.23333333333333334, 0.14285714285714285, 0.5112474437627812, 0.4810126582278481, 0.3105263157894737, 0.5454545454545454, 0.5888157894736842, 0.4791666666666667, 0.625, 0.46735395189003437, 0.29411764705882354, 0.45229007633587787, 0.46601941747572817, 0.6, 0.4696969696969697, 0.7142857142857143, 0.46484375, 0.47619047619047616, 0.43103448275862066, 0.631578947368421, 0.3111111111111111, 0.17647058823529413, 0.46136363636363636, 0.4437869822485207, 0.5311203319502075, ...]",1316,NA


In [39]:
st=StandardScaler()
df[df.columns]=st.fit_transform(df[df.columns])
dftest[dftest.columns]=st.transform(dftest[dftest.columns])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [40]:
df.head()

,impressions,clicks,cost,conversions,ctr
0,-0.346072,-0.372834,-0.189472,-0.256367,-0.818558
1,-0.462041,-0.432742,-0.195043,-0.256367,-1.895981
2,-0.401535,-0.392804,-0.192258,-0.256367,-0.569922
3,-0.441873,-0.392804,-0.189472,-0.256367,1.551773
4,0.778328,0.825321,-0.105211,0.509425,0.302486


In [41]:
dftest.head()

,impressions,clicks,cost,conversions,ctr
0,0.143017,0.056505,-0.154654,-0.064919,-0.150734
1,-0.356156,-0.312927,-0.183205,-0.256367,0.454760
2,-0.441873,-0.402788,-0.191562,-0.256367,0.689834
3,-0.456999,-0.422758,-0.194347,-0.256367,0.258865
4,-0.451957,-0.422758,-0.194347,-0.256367,-0.459417


In [42]:
y=train['revenue']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(3199, 5)
(1372, 5)


In [44]:
model = Sequential()
model.add(Dense(5000, input_dim=5, activation= "relu"))
model.add(Dense(1000, activation= "relu"))
model.add(Dense(500, activation= "relu"))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 5000)              30000     
_________________________________________________________________
dense_9 (Dense)              (None, 1000)              5001000   
_________________________________________________________________
dense_10 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 501       
Total params: 5,532,001
Trainable params: 5,532,001
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
100/100 [==============================] - 7s 63ms/step - loss: 1256714.3963 - mean_squared_error: 1256714.3963
Epoch 2/100
100/100 [==============================] - 6s 62ms/step - loss: 195620.5107 - mean_squared_error: 195620.5107
Epoch 3/100
100/100 [==============================] - 6s 61ms/step - loss: 114605.3010 - mean_squared_error: 114605.3010
Epoch 4/100
100/100 [==============================] - 6s 62ms/step - loss: 223730.3370 - mean_squared_error: 223730.3370
Epoch 5/100
100/100 [==============================] - 6s 61ms/step - loss: 47742.2876 - mean_squared_error: 47742.2876
Epoch 6/100
100/100 [==============================] - 6s 62ms/step - loss: 128741.3786 - mean_squared_error: 128741.3786
Epoch 7/100
100/100 [==============================] - 6s 61ms/step - loss: 55373.6471 - mean_squared_error: 55373.6471
Epoch 8/100
100/100 [==============================] - 6s 62ms/step - loss: 84387.3310 - mean_squared_error: 84387.3310
Epoch 9/100
100/100 [=======

In [46]:
pred_train= model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred= model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred))) 

205.5967582522318
223.13020706498818


In [47]:
test['revenue']=model.predict(dftest)

In [52]:
test['revenue']=[y if y>0 else 0 for y in test.revenue]

In [55]:

test['revenue']=[np.round(y)for y in test.revenue]

In [62]:
np.mean(test['revenue'])

130.88679245283018

In [63]:
sample_sub=pd.read_csv("/content/Sample_Submission.csv")

In [66]:
sample_sub['revenue']=test['revenue']

In [68]:
sample_sub.to_csv("/content/Submission.csv")